# 문서 군집화

문서 군집화(Document Clustering)는 비슷한 텍스트 구성의 문서를 군집화하는 것입니다. 

문서 군집화는 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 앞에서 소개한 텍스트 분류 기반의 문서 분류와 유사합니다. 하지만 텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작합니다. 

In [5]:
import pandas as pd
import glob, os

path = r"/Users/ijiseon/Desktop/ESAA-OB/OpinosisDataset1.0/topics"

#path로 지정한 디렉터리 밑에있는 모든 .data 파일의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path,"*.data"))

In [20]:
filename_list =[]
opinion_text =[]

#개별 파일의 파일명은 filename_list로 취합,
#개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합

for file_ in all_files:
    #개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col = None, header=0, encoding="latin1")
    
    #절대 경로로 주어진 파일명을 가공, 리눅스에서 수행할 때는 다음 \\를 /로 변경
    #맨 마지막 .data 확장자도 제거
    filename_  = file_.split("/")[-1]
    filename = filename_.split(".")[0]
    
    #파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    

In [19]:
pd.read_table(all_files[0], index_col = None, header=0, encoding="latin1").to_string()

"                                                                                                                                                                                                                                                                                               short battery life  I moved up from an 8gb .\n0                                                                                                                                                                                                                                                                                            I love this ipod except for the battery life .\n1                                                                                                                                                                                                                                                                                                           long battery  scratch resistant\n

In [21]:
#파일명 list와 파일 내용 list 객체를 DataFrame으로 생성

document_df = pd.DataFrame({"filename":filename_list, "opinion_text":opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...
3,location_holiday_inn_london,...
4,staff_bestwestern_hotel_sfo,...


> 각 파일 이름(filename) 자체만으로 의견(opinion)의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있습니다. 

> 문서를 TF-IDF 형태로 피처 벡터화하겠습니다. tokenizer는 이전 예제에서 Lemmatization을 구현한 LemNormalize()함수를 이용할 것이며 ngram은 (1,2)로 하고, min_df와 max_df 범위를 설정해 피처의 개수를 제한하겠습니다. 

> TfidVectorizer의 fit_transform()의 인자로 document_df DataFrame의 opinion_text 칼럼을 입력하면 개별 문서 텍스트에 대해 TF_IDF 변환된 피처 벡터화된 행렬을 구할 수 있습니다.

In [22]:
from nltk.stem import WordNetLemmatizer
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /Users/ijiseon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ijiseon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english",
                          ngram_range=(1,2), min_df=0.05, max_df=0.85)

#opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df["opinion_text"])

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


> 문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인해보겠습니다. 군집화 기법은 K-평균을 적용하겠습니다. 

In [28]:
from sklearn.cluster import KMeans

#5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [29]:
document_df["cluster_label"] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,3
1,gas_mileage_toyota_camry_2007,...,0
2,room_holiday_inn_london,...,2
3,location_holiday_inn_london,...,4
4,staff_bestwestern_hotel_sfo,...,4


In [30]:
document_df[document_df["cluster_label"]==0].sort_values(by="filename")

,filename,opinion_text,cluster_label
18,comfort_honda_accord_2008,...,0
43,comfort_toyota_camry_2007,...,0
1,gas_mileage_toyota_camry_2007,...,0
45,interior_honda_accord_2008,...,0
22,interior_toyota_camry_2007,...,0
35,mileage_honda_accord_2008,...,0
47,performance_honda_accord_2008,...,0
42,quality_toyota_camry_2007,...,0
29,seats_honda_accord_2008,...,0
23,transmission_toyota_camry_2007,...,0


In [31]:
document_df[document_df["cluster_label"]==1].sort_values(by="filename")

,filename,opinion_text,cluster_label
33,accuracy_garmin_nuvi_255W_gps,...,1
26,buttons_amazon_kindle,...,1
34,directions_garmin_nuvi_255W_gps,...,1
48,display_garmin_nuvi_255W_gps,...,1
36,eyesight-issues_amazon_kindle,...,1
21,features_windows7,...,1
44,fonts_amazon_kindle,...,1
38,navigation_amazon_kindle,...,1
41,price_amazon_kindle,...,1
10,satellite_garmin_nuvi_255W_gps,...,1


In [34]:
document_df[document_df["cluster_label"]==2].sort_values(by="filename")

,filename,opinion_text,cluster_label
31,bathroom_bestwestern_hotel_sfo,...,2
2,room_holiday_inn_london,...,2
46,rooms_bestwestern_hotel_sfo,...,2
30,rooms_swissotel_chicago,...,2


In [32]:
document_df[document_df["cluster_label"]==3].sort_values(by="filename")

,filename,opinion_text,cluster_label
9,battery-life_amazon_kindle,...,3
0,battery-life_ipod_nano_8gb,...,3
11,battery-life_netbook_1005ha,...,3
12,keyboard_netbook_1005ha,...,3
15,performance_netbook_1005ha,...,3
25,screen_ipod_nano_8gb,...,3
37,screen_netbook_1005ha,...,3
7,size_asus_netbook_1005ha,...,3
24,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,3
14,video_ipod_nano_8gb,...,3


In [33]:
document_df[document_df["cluster_label"]==4].sort_values(by="filename")

,filename,opinion_text,cluster_label
17,food_holiday_inn_london,...,4
32,food_swissotel_chicago,...,4
49,free_bestwestern_hotel_sfo,...,4
39,location_bestwestern_hotel_sfo,...,4
3,location_holiday_inn_london,...,4
50,parking_bestwestern_hotel_sfo,...,4
28,price_holiday_inn_london,...,4
16,service_bestwestern_hotel_sfo,...,4
27,service_holiday_inn_london,...,4
13,service_swissotel_hotel_chicago,...,4


In [35]:
from sklearn.cluster import KMeans

#5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

#소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df["cluster_label"] = cluster_label
document_df.sort_values(by="cluster_label")

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,filename,opinion_text,cluster_label
50,parking_bestwestern_hotel_sfo,...,0
27,service_holiday_inn_london,...,0
28,price_holiday_inn_london,...,0
30,rooms_swissotel_chicago,...,0
20,staff_swissotel_chicago,...,0
31,bathroom_bestwestern_hotel_sfo,...,0
32,food_swissotel_chicago,...,0
17,food_holiday_inn_london,...,0
16,service_bestwestern_hotel_sfo,...,0
13,service_swissotel_hotel_chicago,...,0


## 군집별 핵심 단어 추출하기 

각 군집에 속한 문서는 핵심 단어를 주축으로 군집화돼 있을 것입니다. 이번에는 각 군집을 구성하는 핵심 단어가 어떤 것이 있는지 확인해 보겠습니다. 

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 clusters_centers_ 라는 속성으로 제공합니다. cluster_centers_는 배열 값으로 제공되며, 행은 개별 군집을, 열은 개별 피처를 의미합니다. 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값입니다. 

예를 들어 cluster_centers[0,1]은 0번 군집에서 두 번째 피처의 위치 값입니다. 바로 앞 예제에서 군집 3개로 생성한 KMeans 객체인 km_cluster에서 cluster_centers_ 속성값을 가져온 뒤 값을 확인해보겠습니다

In [36]:
cluster_centers = km_cluster.cluster_centers_
print("cluster_centers shape: ", cluster_centers.shape)
print(cluster_centers)

cluster_centers shape:  (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


> 군집이 3개 word 피처가 4611개 

> 각 행의 배열 값은 각 군집 내의 워드 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것. 0에서 1가지의 값을 가질 수 있으며 1에 가까울수록 중심과 가까운 값을 의미

> **cluster_centers_ 속성 값을 이용해 각 군집별 핵심단어를 찾아보겠습니다**

cluster_centers_ 속성은 넘파이 ndarray입니다. ndarray의 argsort()[:,::-1]을 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값을 반환합니다 (큰 값으로 정렬한 값을 반환하는 게 아니라 큰 값을 가진 배열 내 위치 인덱스 값을 반환하는 것입니다)

이 위치 인덱스 값이 필요한 이유는 핵심 단어 피처의 이름을 출력하기 위해서 입니다. 
<- get_cluster_details()함수가 이에 대한 처리를 담당 

cluster_centers_ 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤 해당 인덱스를 이용해 핵심 단어 이름과 그 때의 상대 위치 값을 추출해 cluster_details라는 Dict 객체 변수에 기록하고 반환하는 것이 get_cluster_details() 함수의 주요 로직입니다

In [57]:
km_cluster.cluster_centers_.argsort()[:,::-1]

array([[3466, 2095, 3648, ..., 2272, 2275,    0],
       [3572,  685, 2342, ..., 1750, 3647, 4610],
       [2252, 3612, 2706, ..., 2791, 2789,    0]])

- 각 클러스터 내 워드 피처 중요도 값이 큰 순으로 **인덱스** 정렬

In [37]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함

def get_cluster_details(cluster_model, cluster_data, 
                        feature_names, clusters_num, top_n_features=10):
    cluster_details={}
    
    #cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
    #군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        #개별 군집별 정보를 담을 데이터 초기화 
        cluster_details[cluster_num]={}
        cluster_details[cluster_num]["cluster"]=cluster_num
        
        #cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        
        #top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, 
                                                            top_feature_indexes].tolist()
        
        #cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]["top_features"]=top_features
        cluster_details[cluster_num]["top_features_value"]=top_feature_values
        filenames=cluster_data[cluster_data["cluster_label"]==cluster_num]["filename"]
        filenames=filenames.values.tolist()
        
        cluster_details[cluster_num]["filenames"]=filenames
        
    return cluster_details

In [56]:
centroid_feature_ordered_ind = km_cluster.cluster_centers_.argsort()[:,::-1]
centroid_feature_ordered_ind[2, :10]

array([2252, 3612, 2706, 1069, 1806, 1807, 4154,  896, 3022, 3241])

- 두번째 클러스터 내 상위 10개 워드 피처를 가져오기 위한 최종 인덱스 

In [59]:
top_features = [feature_names[ind] for ind in centroid_feature_ordered_ind[2, :10]]

In [62]:
feature_names = tfidf_vect.get_feature_names_out()
feature_names.shape

(4611,)

- 위 코드는 TfidfVectorizer 객체인 tfidf_vect에서 피처 이름을 추출하는 데 사용. 이 이름들은 벡터화된 데이터의 각 열을 나타내며, 주어진 문서의 단어 또는 텍스트 피처를 나타냄 

- feature_names에서 앞선 추출한 인덱스를 바탕으로 뽑음 

In [60]:
top_features 

['interior',
 'seat',
 'mileage',
 'comfortable',
 'gas',
 'gas mileage',
 'transmission',
 'car',
 'performance',
 'quality']

In [51]:
cluster_details[2]

{'cluster': 2,
 'top_features': ['interior',
  'seat',
  'mileage',
  'comfortable',
  'gas',
  'gas mileage',
  'transmission',
  'car',
  'performance',
  'quality'],
 'top_features_value': [0.227804691683838,
  0.19395120347901418,
  0.17730089408820415,
  0.12790439933840927,
  0.12265896733770874,
  0.1165812335567078,
  0.10486543788164421,
  0.10152569886339882,
  0.09562358109597213,
  0.09233847022967973],
 'filenames': ['gas_mileage_toyota_camry_2007',
  'comfort_honda_accord_2008',
  'interior_toyota_camry_2007',
  'transmission_toyota_camry_2007',
  'seats_honda_accord_2008',
  'mileage_honda_accord_2008',
  'quality_toyota_camry_2007',
  'comfort_toyota_camry_2007',
  'interior_honda_accord_2008',
  'performance_honda_accord_2008']}

In [44]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print("################ Cluster {0}".format(cluster_num))
        print("Top features:", cluster_detail["top_features"])
        print("Reviews 파일명:", cluster_detail["filenames"][:7])
        print("==========================================================================================")

In [45]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster,cluster_data=document_df,
                                     feature_names=feature_names, clusters_num=3, top_n_features=10)

In [46]:
print_cluster_details(cluster_details)

################ Cluster 0
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명: ['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swissotel_hotel_chicago', 'service_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'staff_swissotel_chicago']
################ Cluster 1
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명: ['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'satellite_garmin_nuvi_255W_gps']
################ Cluster 2
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_20